In [ ]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import cv2
from skimage.transform import rotate

In [ ]:
CELLPHONE_IMG_PATH = os.path.join(os.getcwd(), 'cellphone_imgs')
DB_PATH = os.path.join(os.getcwd(), 'png_imgs')

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

## Generate database

In [ ]:
png_paths = glob.glob(os.path.join(DB_PATH, '*.png'))

In [ ]:
db = {}
for i in range(len(png_paths)):
    img = cv2.imread(png_paths[i], 0)
    kp, des = sift.detectAndCompute(img, None)
    db[png_paths[i]] = (kp, des)
    print(
        "Finish computing SIFT descriptor {:}/{:}".format(
            i + 1, len(png_paths)),
        file=sys.stderr)

## Get query

In [ ]:
query_paths = sorted(glob.glob(os.path.join(CELLPHONE_IMG_PATH, '*.jpg')))

In [ ]:
img = cv2.imread(query_paths[0], 0)
plt.figure(figsize=(20, 20))
plt.imshow(img)

In [ ]:
df = pd.read_csv(os.path.join(os.getcwd(), 'groundtruth.csv'))
groundtruth = {}

In [ ]:
for (cellphone_img, sheet_img) in df.values:
    groundtruth[cellphone_img] = sheet_img + '.png'

## Search

In [ ]:
%lsmagic

In [ ]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

In [ ]:
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [ ]:
def search(db_png_paths,
           des_query,
           expected_match,
           threshold_npairs=25,
           verbose=False):
    scoreList = []
    for idx in range(len(db_png_paths)):
        ref_path = db_png_paths[idx]
        matches = flann.knnMatch(db[ref_path][1], des_query, k=2)

        totalDistance = 0
        counterGood = 0
        # ratio test as per Lowe's paper
        for i, (m, n) in enumerate(matches):
            if m.distance < 0.7 * n.distance:
                counterGood += 1
                totalDistance += m.distance

        scoreList.append({
            'path': ref_path,
            'distance': totalDistance,
            'n_pairs': counterGood,
        })

        if verbose:
            print(
                "Finish searching {:}/{:} distance = {:} (# good pairs = {:})".
                format(idx + 1, len(png_paths), totalDistance, counterGood),
                file=sys.stderr)
    filteredScore = [
        score for score in scoreList if score['n_pairs'] > threshold_npairs
    ]

    sortedScore = sorted(filteredScore, key=lambda x: x['distance'])
    for score in scoreList:
        if score['n_pairs'] <= threshold_npairs:
            sortedScore.append(score)

    rank = 1
    for score in sortedScore:
        if (os.path.split(score['path'])[1] == expected_match):
            return rank
        rank += 1

    print("Expected match = {:}".format(expected_match), file=sys.stderr)
    print(sortedScore, file=sys.stderr)
    raise ValueError("not found")

In [ ]:
MRR = 0
top1acc = 0
query_num = 0
for query_path in query_paths:
    img_query = cv2.imread(query_path, 0)
    kp_query, des_query = sift.detectAndCompute(img, None)
    expected_match = groundtruth[os.path.split(query_path)[1]]
    rank = search(png_paths, des_query, expected_match)

    MRR += (1 / len(query_paths)) * (1 / rank)
    top1acc += (1 / len(query_paths)) * (rank == 1)

    query_num += 1
    print("Query {:} : rank = {:}".format(query_num, rank), file=sys.stderr)

In [ ]:
print(("MRR = {:}".format(MRR)))
print(("Top-1 accuracy = {:}".format(top1acc)))


| Experiment               | MRR           | top-1 accuracy  |
| --------------           | ------------- | --------------- |
| SIFT (0 threshold)       | 0.01          | 0               |
| SIFT (50 threshold)      | 0.05          | 0.025           |
| SIFT (100 threshold)     | 0.03          | 0               |

## Visualization

From the search in section earlier, we found that, based on the matching algorithm, the expected musical score has 7th rank out of 210 scores.

In [ ]:
# Visualize the expected musical score
expected_path = os.path.join(DB_PATH, '105370_page_93.png')
expected_img = cv2.imread(expected_path, 0)
plt.figure(figsize=(20, 20))
plt.imshow(expected_img)
plt.show()

The following is the visualization of the matching between the query and the expected score from the database.

In [ ]:
img_db = cv2.imread(expected_path, 0)
img_query = cv2.imread(query_path, 0)
matches = flann.knnMatch(db[expected_path][1], des_query, k=2)

In [ ]:
# Need to draw only good matches, so create a mask
matchesMask = [[0, 0] for i in range(len(matches))]
counterGood = 0
# ratio test as per Lowe's paper
for i, (m, n) in enumerate(matches):
    if m.distance < 0.7 * n.distance:
        matchesMask[i] = [1, 0]
        counterGood += 1
print(("There are {:} good matched pairs".format(counterGood)))

In [ ]:
draw_params = dict(
    matchColor=(0, 255, 0),
    singlePointColor=(255, 0, 0),
    matchesMask=matchesMask,
    flags=0)

In [ ]:
img3 = cv2.drawMatchesKnn(img_db, db[expected_path][0], img_query, kp_query,
                          matches, None, **draw_params)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img3)
plt.show()